In [1]:
import numpy as np
from scipy import linalg
from scipy.sparse.linalg import eigsh
from scipy import sparse

hamiltonian = np.zeros((6240,6240))

with open('datafile','r') as data:
    count = 0
    while True:
        line = data.readline()
        if count == 0:
            count += 1
        if not line:
            break
        else:
            values = line.split()
            hamiltonian[int(values[0])-1][int(values[1])-1] = float(values[2])
            hamiltonian[int(values[1])-1][int(values[0])-1] = float(values[2])

In [2]:
size = 100
sub_hamiltonian = hamiltonian[0:size,0:size] # used 100 particles to work with
sparse_sub_hamiltonian = sparse.csr_matrix(sub_hamiltonian)
eigenvalues, eigenvectors = eigsh(sparse_sub_hamiltonian,99,which = 'LM')
print sum(eigenvalues) # should be -8234.4466281 

-8234.4466281


In [3]:
# arbitrarily set one of the determinants to 1.
psi_initial = np.zeros(shape=(size, 1), dtype = complex)
psi_initial[0] = 1 # I set the first determinant to 1

In [23]:
from scipy.integrate import ode

y0 = psi_initial
t0 = 0

def f(t, y, scaling_factor):
    """
    TDSE: dy/dt = (-i/h_bar)*H*y 
    Blows up when scaled by 1/h_bar so it has been scaled by scaling factor
    Made dy/dt real by omitting complex i
    """
    global sub_hamiltonian
    y_out = ((-1.0)*np.matmul(sub_hamiltonian,y))/(scaling_factor)
    return y_out

def runge_kutta_order8(t1,dt,constant):
    """
    Explicit Runge-Kutta method of order 8(5,3) by Dormand and Prince
    Returns a tuple of the final time and wavefunction
    """
    global y0,t0
    r = ode(f).set_integrator('dop853').set_f_params(constant) # also 8th order
    r.set_initial_value(y0,t0)
    while r.successful() and r.t < t1:
        r.integrate(r.t+dt)
    return r.t, r.y

def runge_kutta_order5(t1,dt,constant):
    """
    Explicit Runge-Kutta method of order 5(4) by Dormand and Prince
    Returns a tuple of the final time and wavefunction
    """
    global y0,t0
    r = ode(f).set_integrator('dopri5').set_f_params(constant) # also 5th order
    r.set_initial_value(y0,t0)
    while r.successful() and r.t < t1:
        r.integrate(r.t+dt)
    return r.t,r.y

def runge_kutta_order8_stepwise(t1,dt,constant):
    """
    The stepwise function gives a sense of how the sum of probabilities diverges in each time step
    Returns a tuple of arrays.
    First array is the different steps in time
    Second array is the sum of probabilities at each time step
    """
    global y0,t0
    sum_probabilities = []
    time = []
    r = ode(f).set_integrator('dop853').set_f_params(constant) # also 8th order
    r.set_initial_value(y0,t0)
    while r.successful() and r.t < t1:
        r.integrate(r.t+dt)
        time.append(r.t)
        sum_probabilities.append(sum(np.absolute(r.y)**2))
    return time, sum_probabilities

def runge_kutta_order5_stepwise(t1,dt,constant):
    """
    The stepwise function gives a sense of how the sum of probabilities diverges in each time step
    Returns a tuple of arrays.
    First array is the different steps in time
    Second array is the sum of probabilities at each time step
    """
    global y0,t0
    sum_probabilities = []
    time = []
    r = ode(f).set_integrator('dopri5').set_f_params(constant) # also 8th order
    r.set_initial_value(y0,t0)
    while r.successful() and r.t < t1:
        r.integrate(r.t+dt)
        time.append(r.t)
        sum_probabilities.append(sum(np.absolute(r.y)**2))
    return time, sum_probabilities

In [27]:
# fixed the scaling factor and total time at 10s. dt is varied from 10^(-3) to 1.
# In spite of this, there is not much change in the divergence of probabilities or a clear pattern
for i in range(-3,1):
    dt = 10**i
    t, y = runge_kutta_order8(1,dt,10**3)
    print sum(np.absolute(y)**2)

[ 1.19960315]
[ 1.19960315]
[ 1.22163472]
[ 1.19960315]


In [ ]:
import matplotlib.pyplot as plt
t,y = runge_kutta_order8_stepwise(10,1)
plt.plot(t,y,'x')
plt.show()